In [1]:
#Auxiliares
import os
import pickle
from time import time
from datetime import datetime
from json import dumps

#Dados
import pandas as pd

#preprocessing and transformation
from sklearn.preprocessing import MaxAbsScaler

#Machine learning
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

#Metrics
from sklearn.metrics import classification_report

seed = 42

In [3]:
#Variables
base_path = 'D:/03. Documentos/Mestrado/22032020 - Experimentos/05. Organizado/03. Datasets/Atendimento-Balanced-Multiclass'
save_path = 'output'

data='req-text'
label='Atendimento'

x_train_file = 'X_train.csv'
y_train_file = 'y_train.csv'
x_test_file = 'X_test.csv'
y_test_file = 'y_test.csv'

#Load data
X_train = pd.read_csv(os.path.join(base_path, x_train_file), sep=';', encoding='ansi')
y_train = pd.read_csv(os.path.join(base_path, y_train_file), sep=';', encoding='ansi')
X_test = pd.read_csv(os.path.join(base_path, x_test_file), sep=';', encoding='ansi')
y_test = pd.read_csv(os.path.join(base_path, y_test_file), sep=';', encoding='ansi')

#Checking on data
print(X_train.columns)
print(X_train.shape)
print(y_train.Atendimento.value_counts())
print(y_test.Atendimento.value_counts())

Index(['pid', 'req-text', 'resp-text', '1funct-request', '2pronoun-request',
       '3ppron-request', '4i-request', '5we-request', '6you-request',
       '7shehe-request',
       ...
       '58home-response', '59money-response', '60relig-response',
       '61death-response', '62assent-response', '63nonfl-response',
       '64filler-response', 'Clareza', 'Atendimento', 'tempo_resposta'],
      dtype='object', length=134)
(6982, 134)
0    2367
1    2319
2    2296
Name: Atendimento, dtype: int64
2    1029
1    1006
0     958
Name: Atendimento, dtype: int64


In [4]:
# Define pipeline and GridSearch CV

params = {
    'vect_analyzer': 'word',
    'vect_max_features': None,
    'vect_min_df': 1,
    'vect_max_df': 0.75,
    'vect_ngram_range': (1,2),
    'tfidf_use_idf': True,
    'tf_idf_norm': 'l2',
    'clf_solver':'lbfgs',
    'clf_max_iter': 10000,
    'clf_C': 0.1,
    'gs_cv': 10,
    'gs_scoring': 'f1_macro'   
}

pipelineWord = Pipeline([
    ('vect',   CountVectorizer(analyzer=params.get('vect_analyzer'),
                               max_features=params.get('vect_max_features'),
                               min_df=params.get('vect_min_df'),
                               max_df=params.get('vect_max_df'),
                               ngram_range=params.get('vect_ngram_range'))),
    
    ('tfidf', TfidfTransformer(use_idf=params.get('tfidf_use_idf'),
                               norm=params.get('tf_idf_norm'))),
    
    ('scaler', MaxAbsScaler()),
    
    ('clf', LogisticRegression(random_state=seed,
                               n_jobs=6,
                               C=params.get('clf_C'),
                               solver=params.get('clf_solver'),
                               max_iter=params.get('clf_max_iter')))
])

gs_parameters = {
    #'vect__min_df': (1, 0.1, 0.25),
    #'vect__max_df': (1.0, 0.75),
    #'clf__C': (0.1, 1, 10, 100)
}

# Define grid search
grid_search_word = GridSearchCV(pipelineWord,
                               gs_parameters,
                               cv=params.get('gs_cv'),
                               scoring=params.get('gs_scoring'),
                               n_jobs=6,
                               verbose=10
                               )

In [5]:
X = X_train[data]
Y = y_train[label]
x = X_test[data]
y = y_test[label]

In [26]:
print('Quantidade de características extraídas:\n')
print(CountVectorizer(analyzer=params.get('vect_analyzer'),
                               max_features=params.get('vect_max_features'),
                               min_df=params.get('vect_min_df'),
                               max_df=params.get('vect_max_df'),
                               ngram_range=params.get('vect_ngram_range')).fit_transform(x).shape)

Quantidade de características extraídas:

(2993, 118264)


In [8]:
print("Executando Gridsearch para Solicitacoes - Multiclass WORD - Classe Atendimento")

now = str(datetime.now()).split('.')[0].replace('-', '_').replace(' ', '_').replace(':', '_')
print(now)

t0 = time()
grid_search_word.fit(X, Y)
print("done in %0.3fs" % (time() - t0))
print("Best score: %0.3f" % grid_search_word.best_score_)
print("Best parameters set:")
best_parameters = grid_search_word.best_estimator_.get_params()
for param_name in sorted(gs_parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Executando Gridsearch para Solicitacoes - Multiclass WORD - Classe Atendimento
2020_05_19_23_22_47
Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of  10 | elapsed:   21.5s remaining:   50.4s
[Parallel(n_jobs=6)]: Done   5 out of  10 | elapsed:   21.8s remaining:   21.8s
[Parallel(n_jobs=6)]: Done   7 out of  10 | elapsed:   34.7s remaining:   14.8s
[Parallel(n_jobs=6)]: Done  10 out of  10 | elapsed:   35.6s finished


done in 43.381s
Best score: 0.446
Best parameters set:


In [9]:
# Saving Model
f_save = now + '_Atendimento_Solicitacao_Multiclass_Balanced_word.sav'
pickle.dump(grid_search_word, open(os.path.join(save_path, f_save),'wb'))

# Saving Parameters
with open(os.path.join(save_path, 'params.txt'),'a') as f:
    f.write('\n\n' + ('#'*60))
    f.write('\n'+f_save + '\n\n')
    f.write('Parameters:\n')
    f.write(dumps(params) + '\n')
    f.write('\nGridSearch Best Parameters:\n')
    for param_name in sorted(gs_parameters.keys()):        
        f.write("%s: %r" % (param_name, best_parameters[param_name]) + '\n')    

# Validation

In [10]:
# Predictions
y_pred_train = grid_search_word.predict(X)
y_pred_test = grid_search_word.predict(x)

In [11]:
print('#'*50)
print('Report for TRAIN')
print('#'*50)
print(classification_report(Y, y_pred_train))

print('#'*50)
print('Report for TEST')
print('#'*50)
print(classification_report(y, y_pred_test))

##################################################
Report for TRAIN
##################################################
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      2367
           1       0.96      0.96      0.96      2319
           2       0.95      0.97      0.96      2296

    accuracy                           0.97      6982
   macro avg       0.97      0.97      0.97      6982
weighted avg       0.97      0.97      0.97      6982

##################################################
Report for TEST
##################################################
              precision    recall  f1-score   support

           0       0.42      0.47      0.45       958
           1       0.41      0.41      0.41      1006
           2       0.50      0.45      0.47      1029

    accuracy                           0.44      2993
   macro avg       0.44      0.44      0.44      2993
weighted avg       0.45      0.44      0.44      2993



In [12]:
with open(os.path.join(save_path, 'params.txt'),'a') as f:
    f.write('\n\n' + ('#'*60))
    f.write('\nReport for TRAIN')
    f.write('\n' + ('#'*60))
    f.write('\n' + classification_report(Y, y_pred_train))
    
    f.write('\n\n' + ('#'*60))
    f.write('\nReport for TEST')
    f.write('\n' + ('#'*60))
    f.write('\n' + classification_report(y, y_pred_test))